## Аналитика цепочки поставок и потерь в розничной торговле

**Applepen** $-$ это большая торговая сеть, которая занимается продажей всего двух продуктов: **яблок** и **карандашей**.
Ее магазины расположены в различных уголках Соединенных Штатов и более 10 лет обслуживают покупателей.

Недавно топ-менеджмент компании решил более активно использовать имеющиеся у них данные в принятии решений.
Каждый магазин собирает информацию о:

1. закупках (поставки яблок и карандашей два раза в месяц),
2. продажах (лог транзакций, по записи на каждую проданную позицию),
3. инвентарь (месячные данные общего количества яблок и карандашей на складе).

#### Импорт необходимых библиотек

In [1]:
from pathlib import Path
from IPython.display import display, Markdown
import pandas as pd
import numpy as np
import csv
import datetime

#### Список всех .csv файлов

In [2]:
folder_path = Path(".")
csv_files = sorted([f.name for f in folder_path.glob("*.csv")])

for file in csv_files:
    display(Markdown(f"**{file}**"))

**MS-b1-inventory.csv**

**MS-b1-sell.csv**

**MS-b1-supply.csv**

**MS-b2-inventory.csv**

**MS-b2-sell.csv**

...

**MS-s4-sell.csv**

**MS-s4-supply.csv**

**MS-s5-inventory.csv**

**MS-s5-sell.csv**

**MS-s5-supply.csv**

#### Обработка файлов

Рассматрим на примере трех файлов (MS-b1-supply.csv, MS-b1-inventory.csv, MS-b1-sell.csv), содержимое по файлам с информацией о закупках, продажах и инвентаре. С помощью цикла считаем каждый файл в отдельный DataFrame и выводим первые 5 строк (метод head()) для первичной проверки содержимого таблиц. Это помогает убедиться, что данные считались корректно и готовы к дальнейшей обработке.

In [3]:
files = {
    'df_supply': 'MS-b1-supply.csv',
    'df_inventory': 'MS-b1-inventory.csv',
    'df_sell': 'MS-b1-sell.csv'
}

dataframes = {}

for name, path in files.items():
    df = pd.read_csv(path)
    dataframes[name] = df
    display(Markdown(f"**Файл {name} (первые 5 строк):**"))
    display(df.head())

**Файл df_supply (первые 5 строк):**

,date,apple,pen
0,2006-01-01,35086,2730
1,2006-01-15,35002,2625
2,2006-02-01,34963,2759
3,2006-02-15,34869,2715
4,2006-03-01,35188,2599


**Файл df_inventory (первые 5 строк):**

,date,apple,pen
0,2006-01-31,12157,811
1,2006-02-28,29859,2280
2,2006-03-31,42135,3317
3,2006-04-30,55235,4094
4,2006-05-31,66963,5158


**Файл df_sell (первые 5 строк):**

,date,sku_num
0,2006-01-01,MS-b1-ap-48914c5b-14d2-4b20-bdaf-b2ff5d9f4f0c
1,2006-01-01,MS-b1-ap-6baf7287-3e6a-4728-a3b1-8613de51eef8
2,2006-01-01,MS-b1-ap-83d7b005-c7d9-4deb-93a2-a8f7606d02b5
3,2006-01-01,MS-b1-ap-9099c5de-028d-4b07-97be-6f6532b8674a
4,2006-01-01,MS-b1-ap-ee0ce843-a979-4798-a542-75a41d1610c3


### Задание 1. Состояние склада на каждый день
Данные о состоянии склада в конце каждого дня после того как все поставки и продажы были совершены.
Подобная информация будет очень ценна менеджерам магазинов.
Состояние склада должно строится на основне месячных данных об инвентаре.
Известно, что люди воруют из магазинов, но сейчас нет никакой возможности узнать объем сворованного товара.

#### Выбираем нужный магазин

Функция load_store_data загружает CSV-файлы для заданного магазина (через его код в названии файла).  
Аргумент:
store_code (str): код магазина, например 'b1', 'm2', 's4'  
Возвращает:  
df_supply, df_inventory, df_sell — три DataFrame

In [4]:
def load_store_data(store_code):
    prefix = f'MS-{store_code}'
    df_supply = pd.read_csv(f'{prefix}-supply.csv')
    df_inventory = pd.read_csv(f'{prefix}-inventory.csv')
    df_sell = pd.read_csv(f'{prefix}-sell.csv')
    
    return df_supply, df_inventory, df_sell

Для примера возьмем магазин с кодом b1. Проверим, что нужные файлы загрузились (выводим первые 5 строчек).

In [5]:
store = 'b1'

df_supply, df_inventory, df_sell = load_store_data(store)

display(df_supply.head())
display(df_inventory.head())
display(df_sell.head())

,date,apple,pen
0,2006-01-01,35086,2730
1,2006-01-15,35002,2625
2,2006-02-01,34963,2759
3,2006-02-15,34869,2715
4,2006-03-01,35188,2599


,date,apple,pen
0,2006-01-31,12157,811
1,2006-02-28,29859,2280
2,2006-03-31,42135,3317
3,2006-04-30,55235,4094
4,2006-05-31,66963,5158


,date,sku_num
0,2006-01-01,MS-b1-ap-48914c5b-14d2-4b20-bdaf-b2ff5d9f4f0c
1,2006-01-01,MS-b1-ap-6baf7287-3e6a-4728-a3b1-8613de51eef8
2,2006-01-01,MS-b1-ap-83d7b005-c7d9-4deb-93a2-a8f7606d02b5
3,2006-01-01,MS-b1-ap-9099c5de-028d-4b07-97be-6f6532b8674a
4,2006-01-01,MS-b1-ap-ee0ce843-a979-4798-a542-75a41d1610c3


Функция calculate_daily_inventory рассчитывает ежедневное состояние склада магазина на основе данных о поставках, продажах и инвентаре.   
Для этого она использует данные о продажах и поставках. Каждый день:  
	•	из остатков вычитаются проданные товары,  
	•	прибавляются новые поставки (если были),  
	•	затем сохраняется итоговое количество на складе.  
В конце получается таблица, где указано, сколько товара было на складе в каждый день.

In [6]:
def calculate_daily_inventory(df_supply, df_inventory, df_sell):
    apple_stock = 0
    pen_stock = 0

    result_rows = []

    prev_date = df_sell.iloc[0]['date']

    for sale in df_sell.itertuples(index=False):
        cur_date, sku = sale.date, sale.sku_num
        item_code = sku[6:8]  # код товара, 'ap' или 'pe', яблоки или карандаши

        if cur_date != prev_date:
            daily_supply = df_supply[df_supply['date'] == prev_date]
            if not daily_supply.empty:
                apple_stock += int(daily_supply.iloc[0]['apple'])
                pen_stock += int(daily_supply.iloc[0]['pen'])

            result_rows.append([prev_date, apple_stock, pen_stock])
            prev_date = cur_date

        if item_code == 'ap':
            apple_stock -= 1
        elif item_code == 'pe':
            pen_stock -= 1

    
    daily_supply = df_supply[df_supply['date'] == cur_date]
    if not daily_supply.empty:
        apple_stock += int(daily_supply.iloc[0]['apple'])
        pen_stock += int(daily_supply.iloc[0]['pen'])

    result_rows.append([cur_date, apple_stock, pen_stock])

    result_df = pd.DataFrame(result_rows, columns=['date', 'apple', 'pen'])
    return result_df

Применим функцию для магазина b1:

In [7]:
daily_inventory_b1 = calculate_daily_inventory(df_supply, df_inventory, df_sell)
daily_inventory_b1.head()

,date,apple,pen
0,2006-01-01,33271,2574
1,2006-01-02,31409,2431
2,2006-01-03,29529,2260
3,2006-01-04,27732,2107
4,2006-01-05,25790,1974


### Задание 2. Месячные данные о количестве сворованного товара

Функция get_stolen считает, сколько товара (отдельно яблок и карандашей) было украдено каждый месяц из выбранного магазина (выше выбран для примера магазин b1).

In [8]:
def get_stolen(df_inventory, df_daily):

    stolen_rows = []
    stolen_apples_total = 0
    stolen_pens_total = 0

    for row in df_inventory.itertuples(index=False):
        date = row.date
        expected_apples = row.apple
        expected_pens = row.pen

        actual = df_daily[df_daily.date == date]

        if not actual.empty:
            real_apples = int(actual.iloc[0].apple)
            real_pens = int(actual.iloc[0].pen)
        else:
            real_apples = 0
            real_pens = 0

        stolen_apples = real_apples - expected_apples - stolen_apples_total
        stolen_pens = real_pens - expected_pens - stolen_pens_total

        stolen_rows.append([date, stolen_apples, stolen_pens])

        stolen_apples_total += stolen_apples
        stolen_pens_total += stolen_pens

    return pd.DataFrame(stolen_rows, columns=['date', 'apple', 'pen'])

In [9]:
stolen_b1 = get_stolen(df_inventory, daily_inventory_b1)

stolen_b1.head()

,date,apple,pen
0,2006-01-31,10,11
1,2006-02-28,6,6
2,2006-03-31,7,6
3,2006-04-30,6,14
4,2006-05-31,8,1


### Задание 3. Агрегированные данные об объемах продаж и количестве сворованной продукции по штату и году

Сначала получаем ежедневные остатки и данные о кражах из магазина. Из названия файла извлекаем название штата (state).     
Делаем таблицу с колонками ['year', 'state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen'].   
Код ниже считает, сколько яблок и карандашей было продано и украдено за каждый год для одного магазина. Он использует ранее рассчитанные данные (в функциях выше, Задание 1 и Задание 2): ежедневные остатки и количество краж по месяцам.    
В результате получается таблица с годовой сводкой по продажам и потерям (украдено) товара.

In [10]:
df_daily = calculate_daily_inventory(df_supply, df_inventory, df_sell)
df_stolen = get_stolen(df_inventory, df_daily)

In [11]:
state = df_sell.iloc[0].sku_num[:2]

rows = []
cols = ['year', 'state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen']
cur_year = int(df_supply.iloc[0]['date'][:4])

In [12]:
a_sup, p_sup = 0, 0

for i, row in enumerate(df_supply.itertuples(index=False)):
    a_sup += row.apple
    p_sup += row.pen

    if i % 24 == 23:
        y_end = f'{cur_year}-12-31'
        daily = df_daily[df_daily.date == y_end]
        inv = df_inventory[df_inventory.date == y_end]
        stolen = df_stolen[df_stolen.date == y_end]

        if not daily.empty and not inv.empty and not stolen.empty:
            a_sold = a_sup - int(daily.iloc[0].apple)
            p_sold = p_sup - int(daily.iloc[0].pen)
            a_stolen = int(stolen.iloc[0].apple)
            p_stolen = int(stolen.iloc[0].pen)

            rows.append([cur_year, state, a_sold, a_stolen, p_sold, p_stolen])

        cur_year += 1

year_df = pd.DataFrame(rows, columns=cols)
year_df.head()

,year,state,apple_sold,apple_stolen,pen_sold,pen_stolen
0,2006,MS,681341,8,52555,7
1,2007,MS,1362622,12,104494,8
2,2008,MS,2047732,6,156526,5
3,2009,MS,2730028,8,208937,7
4,2010,MS,3410439,5,261537,12
